# Towards a Conflict Heuristic (DH 2023)

## 07. Novel Ranking Average Values

Last updated: 10.07.2023

julian.haeussler[at]tu-darmstadt.de

In [1]:
# imports

import csv
import glob
import math
import os
import pandas as pd
import pickle
import re

In [2]:
# load data

# verb phrases + values

df_VPs_Romantik_core = pd.read_csv('../Analyseergebnisse/csv/230420_df_VPs_Romantik_core.csv', index_col=0)

In [3]:
df_VPs_Realismus = pd.read_csv('../Analyseergebnisse/csv/230420_df_VPs_Realismus.csv', index_col=0)

In [4]:
df_VPs_Naturalismus = pd.read_csv('../Analyseergebnisse/csv/230420_df_VPs_Naturalismus.csv', index_col=0)

In [5]:
# titles

lst_titles_Romantik = []

lst_files = glob.glob(os.path.join(os.getcwd(), r'../Daten/Romantik (abgeschlossen)/TXT/Kernkorpus', '*.txt'))

for entry in lst_files:
    lst_titles_Romantik.append(re.search(r"(?<=Kernkorpus\\)(.*)(?=.txt)",entry).group(1))

In [6]:
lst_titles_Realismus = []

lst_files = glob.glob(os.path.join(os.getcwd(), r'../Daten/Realismus', '*.txt'))

for entry in lst_files:
    lst_titles_Realismus.append(re.search(r"(?<=Realismus\\)(.*)(?=.txt)",entry).group(1))

In [7]:
lst_titles_Naturalismus = []

lst_files = glob.glob(os.path.join(os.getcwd(), r'../Daten/Naturalismus', '*.txt'))

for entry in lst_files:
    lst_titles_Naturalismus.append(re.search(r"(?<=Naturalismus\\)(.*)(?=.txt)",entry).group(1))

In [8]:
# years

file = r'../Daten/Romantik (abgeschlossen)/Romane der Romantik_selection.xlsx'

df_meta_Romantik = pd.read_excel(file)

lst_yrs_Romantik = df_meta_Romantik['Datum Erstdruck'].tolist()

In [9]:
lst_yrs_Romantik[1] = 1812 # Arnim_Achim_von_Die_Kronenwaechter_Erster_Band
lst_yrs_Romantik[2] = 1817 # Arnim_Achim_von_Die_Kronenwaechter_Zweiter_Band
lst_yrs_Romantik[10] = 1795 # Hoelderlin_Friedrich_Hyperions_Jugend
lst_yrs_Romantik[11] = 1797 # Hoelderlin_Friedrich_Hyperion_oder_der_Eremit_in_Griechenland

In [10]:
file = r'../Daten/Metadaten_Korpus_Realismus_Romane_d-Prose.xlsx'

df_meta_Realismus = pd.read_excel(file)

lst_yrs_Realismus = df_meta_Realismus['verwendetes Datum'].tolist()

In [11]:
file = r'../Daten/Metadaten_Korpus_Naturalismus_Romane_d-Prose.xlsx'

df_meta_Naturalismus = pd.read_excel(file)

lst_yrs_Naturalismus = df_meta_Naturalismus['verwendetes Datum'].tolist()

In [12]:
# create dataframes novel ranking

df_novels_ranked_Romantik_core = pd.DataFrame(columns=['novel','year','mean_aro_adj','mean_val_adj',
                                                       'mean_conf_dornseiff', 'mean_conf_annotation'])

In [13]:
df_novels_ranked_Realismus_Naturalismus = pd.DataFrame(columns=['novel','year','mean_aro_adj','mean_val_adj',
                                                                'mean_conf_dornseiff', 'mean_conf_annotation'])

In [14]:
# fill column novels

df_novels_ranked_Romantik_core['novel'] = lst_titles_Romantik

In [15]:
lst_titles_Realismus_Naturalismus = lst_titles_Realismus.copy()

In [16]:
lst_titles_Realismus_Naturalismus.extend(lst_titles_Naturalismus)

In [17]:
df_novels_ranked_Realismus_Naturalismus['novel'] = lst_titles_Realismus_Naturalismus

In [18]:
# fill column year

df_novels_ranked_Romantik_core['year'] = lst_yrs_Romantik

In [19]:
lst_yrs_Realismus_Naturalismus = lst_yrs_Realismus.copy()

lst_yrs_Realismus_Naturalismus.extend(lst_yrs_Naturalismus)

In [20]:
df_novels_ranked_Realismus_Naturalismus['year'] = lst_yrs_Realismus_Naturalismus

In [21]:
# transform dataframes VPs into dicts

dict_VPs_Romantik_core = df_VPs_Romantik_core.to_dict(orient='records')

In [22]:
df_VPs_Realismus_Naturalismus = df_VPs_Realismus.copy()

In [23]:
df_VPs_Realismus_Naturalismus = df_VPs_Realismus_Naturalismus.append(df_VPs_Naturalismus, ignore_index=True)

C:\Users\haeus\AppData\Local\Temp\ipykernel_13756\1485876350.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_VPs_Realismus_Naturalismus = df_VPs_Realismus_Naturalismus.append(df_VPs_Naturalismus, ignore_index=True)


In [24]:
dict_VPs_Realismus_Naturalismus = df_VPs_Realismus_Naturalismus.to_dict(orient='records')

In [25]:
# fill columns 'mean_aro_adj', 'mean_val_adj', 'mean_conf_dornseiff' and 'mean_conf_annotation'

# Romantik core

lst_mean_aro_adj_Romantik_core = []
lst_mean_val_adj_Romantik_core = []
lst_mean_conf_dornseiff_Romantik_core = []
lst_mean_conf_annotation_Romantik_core = []

for title in lst_titles_Romantik:
    lst_aro_all = []
    lst_val_all = []
    lst_dorn_all = []
    lst_anno_all = []
    for i in range(0,len(dict_VPs_Romantik_core)):
        if dict_VPs_Romantik_core[i]['novel_title'] == title:
            aro = dict_VPs_Romantik_core[i]['mean_aro_adj']
            val = dict_VPs_Romantik_core[i]['mean_val_adj']
            dorn = dict_VPs_Romantik_core[i]['mean_conf_dornseiff']
            anno = dict_VPs_Romantik_core[i]['mean_conf_annotation']
                
            if math.isnan(aro) == False:
                lst_aro_all.append(aro)
            if math.isnan(val) == False:
                lst_val_all.append(val)
            if math.isnan(dorn) == False:
                lst_dorn_all.append(dorn)               
            if math.isnan(anno) == False:
                lst_anno_all.append(anno)
    
    lst_mean_aro_adj_Romantik_core.append(sum(lst_aro_all)/len(lst_aro_all))
    lst_mean_val_adj_Romantik_core.append(sum(lst_val_all)/len(lst_val_all))
    lst_mean_conf_dornseiff_Romantik_core.append(sum(lst_dorn_all)/len(lst_dorn_all))
    lst_mean_conf_annotation_Romantik_core.append(sum(lst_anno_all)/len(lst_anno_all))

In [26]:
print(len(lst_mean_aro_adj_Romantik_core))
print(len(lst_mean_val_adj_Romantik_core))
print(len(lst_mean_conf_dornseiff_Romantik_core))
print(len(lst_mean_conf_annotation_Romantik_core))

26
26
26
26


In [27]:
df_novels_ranked_Romantik_core['mean_aro_adj'] = lst_mean_aro_adj_Romantik_core
df_novels_ranked_Romantik_core['mean_val_adj'] = lst_mean_val_adj_Romantik_core
df_novels_ranked_Romantik_core['mean_conf_dornseiff'] = lst_mean_conf_dornseiff_Romantik_core
df_novels_ranked_Romantik_core['mean_conf_annotation'] = lst_mean_conf_annotation_Romantik_core

In [28]:
df_novels_ranked_Romantik_core.head()

,novel,year,mean_aro_adj,mean_val_adj,mean_conf_dornseiff,mean_conf_annotation
0,"Arnim_Achim_von_Armut,_Reichtum,_Schuld_und_Bu...",1810,0.007470,-0.010451,-0.000126,-0.016061
1,Arnim_Achim_von_Die_Kronenwaechter_Erster_Band,1812,0.005924,-0.011534,0.002519,-0.013823
2,Arnim_Achim_von_Die_Kronenwaechter_Zweiter_Band,1817,0.006887,-0.013096,0.003157,-0.012190
3,Arnim_Achim_von_Hollins_Liebeleben,1802,0.009937,-0.009634,0.000725,-0.016707
4,Brentano_Clemens_Godwi_oder_Das_steinerne_Bild...,1802,0.007211,-0.010161,-0.001509,-0.017165


In [29]:
# Realismus Naturalismus

lst_mean_aro_adj_Realismus_Naturalismus = []
lst_mean_val_adj_Realismus_Naturalismus = []
lst_mean_conf_dornseiff_Realismus_Naturalismus = []
lst_mean_conf_annotation_Realismus_Naturalismus = []

for title in lst_titles_Realismus_Naturalismus:
    lst_aro_all = []
    lst_val_all = []
    lst_dorn_all = []
    lst_anno_all = []
    for i in range(0,len(dict_VPs_Realismus_Naturalismus)):
        if dict_VPs_Realismus_Naturalismus[i]['novel_title'] == title:
            aro = dict_VPs_Realismus_Naturalismus[i]['mean_aro_adj']
            val = dict_VPs_Realismus_Naturalismus[i]['mean_val_adj']
            dorn = dict_VPs_Realismus_Naturalismus[i]['mean_conf_dornseiff']
            anno = dict_VPs_Realismus_Naturalismus[i]['mean_conf_annotation']
                
            if math.isnan(aro) == False:
                lst_aro_all.append(aro)
            if math.isnan(val) == False:
                lst_val_all.append(val)
            if math.isnan(dorn) == False:
                lst_dorn_all.append(dorn)               
            if math.isnan(anno) == False:
                lst_anno_all.append(anno)
    
    lst_mean_aro_adj_Realismus_Naturalismus.append(sum(lst_aro_all)/len(lst_aro_all))
    lst_mean_val_adj_Realismus_Naturalismus.append(sum(lst_val_all)/len(lst_val_all))
    lst_mean_conf_dornseiff_Realismus_Naturalismus.append(sum(lst_dorn_all)/len(lst_dorn_all))
    lst_mean_conf_annotation_Realismus_Naturalismus.append(sum(lst_anno_all)/len(lst_anno_all))

In [30]:
print(len(lst_mean_aro_adj_Realismus_Naturalismus))
print(len(lst_mean_val_adj_Realismus_Naturalismus))
print(len(lst_mean_conf_dornseiff_Realismus_Naturalismus))
print(len(lst_mean_conf_annotation_Realismus_Naturalismus))

102
102
102
102


In [31]:
df_novels_ranked_Realismus_Naturalismus['mean_aro_adj'] = lst_mean_aro_adj_Realismus_Naturalismus
df_novels_ranked_Realismus_Naturalismus['mean_val_adj'] = lst_mean_val_adj_Realismus_Naturalismus
df_novels_ranked_Realismus_Naturalismus['mean_conf_dornseiff'] = lst_mean_conf_dornseiff_Realismus_Naturalismus
df_novels_ranked_Realismus_Naturalismus['mean_conf_annotation'] = lst_mean_conf_annotation_Realismus_Naturalismus

In [32]:
df_novels_ranked_Realismus_Naturalismus.head()

,novel,year,mean_aro_adj,mean_val_adj,mean_conf_dornseiff,mean_conf_annotation
0,Boy-Ed_Ida_Empor,1892,-0.021688,-0.030646,0.008322,0.011659
1,Boy-Ed_Ida_Fanny_Foerster,1889,-0.021767,-0.029994,0.006790,0.011982
2,Dahn_Felix_Attila,1888,-0.022139,-0.042334,0.017154,0.017710
3,Dahn_Felix_Bissula,1884,-0.021859,-0.041443,0.017669,0.017573
4,Dahn_Felix_Ebroin,1897,-0.022618,-0.039548,0.016681,0.015258


In [33]:
# save

df_novels_ranked_Romantik_core.to_csv('../Analyseergebnisse/csv/230710_df_novels_Romantik_core_average_values.csv', encoding='utf-8-sig') 

In [35]:
df_novels_ranked_Realismus_Naturalismus.to_csv('../Analyseergebnisse/csv/230710_df_novels_Realismus_Naturalismus_average_values.csv', encoding='utf-8-sig') 